# Torre de Hanoi - Algoritmo A*
# Introducción a la Inteligencia Artificial

## Problema de la Torre de Hanoi con 5 discos

### Descripción del problema
La Torre de Hanoi es un problema clásico que consiste en mover una torre de discos de diferentes tamaños de una varilla a otra, siguiendo estas reglas:
1. Solo se puede mover un disco a la vez
2. Un disco solo puede colocarse sobre otro disco más grande
3. Objetivo: mover todos los discos de la varilla izquierda (A) a la derecha (C)

### Estado inicial: Todos los discos en la varilla A
### Estado objetivo: Todos los discos en la varilla C

## Importamos las librerias necesarias

In [ ]:
import heapq
import time
import json
from typing import List, Tuple, Optional, Dict, Any
from datetime import datetime
import tracemalloc
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi
from aima_libs.aima import PriorityQueue as AimaPriorityQueue

## Definimos el problema

In [ ]:
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)

problem = ProblemHanoi(initial=initial_state, goal=goal_state)

In [ ]:
class TorreHanoiAStar:
    """
    Implementación del algoritmo A* para resolver la Torre de Hanoi.
    """

    def __init__(self, initial_state: StatesHanoi, goal_state: StatesHanoi, problem: ProblemHanoi, disks_num: int = 5):
        self.disks_num = disks_num
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.problem = problem
        self.explored_nodes = 0
        self.expanded_nodes = 0

    def multifactorial_heuristic(self, node: NodeHanoi) -> int:
        """
        Heurística multifactorial: Combinación de múltiples factores

        Fundamentación:
        1. Discos mal ubicados: Penaliza discos que no están en la varilla objetivo
        2. Discos bloqueados: Penaliza discos que tienen otros encima en varillas incorrectas
        3. Orden incorrecto: Penaliza cuando los discos no están en orden correcto
        4. Distancia de movimiento: Considera qué tan lejos están los discos de su destino

        Esta heurística es admisible porque nunca sobreestima el costo real,
        ya que cada factor representa movimientos mínimos necesarios.
        """
        h = 0

        # Factor 1: Discos que no están en la varilla objetivo (C)
        misplaced_disks = 0
        for i in range(2):  # Varillas A y B
            misplaced_disks += len(node.state.rods[i])

        # Factor 2: Discos bloqueados en varillas incorrectas
        locked_disks = 0
        for i in range(2):  # Solo varillas A y B
            rod = node.state.rods[i]
            for j, disco in enumerate(rod):
                # Si hay discos encima, está bloqueado
                if j < len(rod) - 1:
                    locked_disks += 1

        # Factor 3: Verificar orden en varilla objetivo
        incorrect_order = 0
        rod_c = node.state.rods[2]
        for i in range(len(rod_c)):
            if rod_c[i] != self.disks_num - i:
                incorrect_order += 1

        # Factor 4: Peso por tamaño de disco (discos más grandes son más costosos de mover)
        disks_weight = 0
        for i in range(2):
            for disc in node.state.rods[i]:
                disks_weight += disc * 0.1  # Pequeño peso adicional

        # Combinación de factores
        h = misplaced_disks * 2 + locked_disks + incorrect_order + disks_weight

        return int(h)

    def simple_heuristics(self, node: NodeHanoi) -> int:
        """
        Heurística simple propuesta: -1 por cada disco en posición correcta
        (Convertida a positiva para A*)
        """
        correct_disks = len(node.state.rods[2])  # Discos en varilla C
        return self.disks_num - correct_disks

    def a_estrella(self, use_multifactorial_heuristic: bool = True) -> Tuple[NodeHanoi, dict]:
        """
        Implementación del algoritmo A*.

        Returns:
            - Nodo final alcanzado
            - Diccionario con estadísticas del algoritmo
        """
        print(f"Iniciando A* con heurística {'multifactorial' if use_multifactorial_heuristic else 'simple'}...")
        print(f"Estado inicial: {self.initial_state}")
        print(f"Estado objetivo: {self.goal_state}")
        print("-" * 60)

        start_time = time.time()

        # Inicialización
        heuristic = self.multifactorial_heuristic if use_multifactorial_heuristic else self.simple_heuristics
        root = NodeHanoi(state=initial_state)
        priority_queue = AimaPriorityQueue(order='min', f=lambda node: node.action.cost + heuristic(node))

        priority_queue.append(root)
        self.explored_nodes = 0
        self.expanded_nodes = 0

        while len(priority_queue) != 0:
            # Seleccionar nodo con menor f
            priority_value, current_node = priority_queue.pop()
            self.explored_nodes += 1

            # Verificar si alcanzamos el objetivo
            if current_node.state == self.goal_state:
                total_time = time.time() - start_time

                # Reconstruir camino
                stats = {
                    'movimientos': current_node.depth,
                    'nodos_explorados': self.explored_nodes,
                    'nodos_expandidos': self.expanded_nodes,
                    'tiempo_ejecucion': total_time,
                    'costo_solucion': current_node.path_cost,
                    'heuristica_usada': 'miltifactorial' if use_multifactorial_heuristic else 'simple'
                }

                return current_node, stats

            # Expandir nodos sucesores
            self.expanded_nodes += 1
            for next_node in current_node.expand(problem=problem):
                priority_queue.append(next_node)

        return None, {'error': 'No se encontró solución'}